### Import CSV files to a postgreSQL database - automated

### Contents<a class="anchor" id="contents"></a>

- [1. Isolate csv files in cwd](#section1)
- [2. Create a new directory in cmd](#section2)
- [3. Move csv files into new directory](#section3)
- [4. Create pandas DataFrame from csv file(s)](#section4)
- [5. Import CSV files to a postgreSQL database - automated script](#section5)
    - [5-1. Quick reference](#section5.1)

In [1]:
# Import libraries
import os
import numpy as np
import pandas as pd
import psycopg2 # - access PostgreSQL with Python

### 1. Isolate csv files in cwd<a class="anchor" id="section1"></a>

In [2]:
# Returns contents of the current working directory
display(os.listdir(os.getcwd()))

['.DS_Store',
 'Customer Contracts$.csv',
 'partTwo.ipynb',
 'customer_contracts.csv',
 'partOne.ipynb',
 '.ipynb_checkpoints',
 'Customer Engagements.csv',
 'Customer Demo.csv']

In [3]:
csvFiles = []

for file in os.listdir(os.getcwd()):
    if file.endswith('.csv'):
        csvFiles.append(file)
        
print(csvFiles)

['Customer Contracts$.csv', 'customer_contracts.csv', 'Customer Engagements.csv', 'Customer Demo.csv']


### 2. Create a new directory in cmd<a class="anchor" id="section2"></a>

In [4]:
# Bash command - creates a new folder in the cwd
# - example:
!mkdir directoryExample

In [5]:
# New directory name
datasetDir = 'datasets'

# Error handling
try:
    # Saves the bash command in string format
    mkdir = 'mkdir {0}'.format(datasetDir)
    print(mkdir)
    # Runs the bash command string
    os.system(mkdir)
except:
    pass

mkdir datasets


### 3. Move csv files into new directory<a class="anchor" id="section3"></a>

In [6]:
# Bash command - movees a file into a folder
# - example:
!mv fileName directoryName

mv: rename fileName to directoryName: No such file or directory


In [7]:
print(csvFiles)

['Customer Contracts$.csv', 'customer_contracts.csv', 'Customer Engagements.csv', 'Customer Demo.csv']


In [8]:
# For loop to run the bash command
for csvFile in csvFiles:
    # Saves the bash command in string format
    # - note: seperate the file name with single quotes to run command
    mvfile = "mv '{0}' {1}".format(csvFile, datasetDir)
    print(mvfile)
    # Runs the bash command string
    os.system(mvfile)

mv 'Customer Contracts$.csv' datasets
mv 'customer_contracts.csv' datasets
mv 'Customer Engagements.csv' datasets
mv 'Customer Demo.csv' datasets


### 4. Create pandas DataFrame from csv file(s)<a class="anchor" id="section4"></a>

In [9]:
# step 1. Define pathway to csv file(s)
dataPath = os.getcwd() + '/' + datasetDir +'/'
print(dataPath)

/Users/sam/Desktop/automateDataUpload/datasets/


In [10]:
# step 2. For loop to read csv files into pandas DataFrames

# Dictionary object to store all DataFrames
dfDict = {}
for csvFile in csvFiles:
    # Error handling
    try:
        # key: value
        dfDict[csvFile] = pd.read_csv(dataPath+csvFile)
    except UnicodeDecodeError:
        dfDict[csvFile] = pd.read_csv(dataPath+csvFile, encoding="ISO-8859-1")
    print(csvFile)

Customer Contracts$.csv
customer_contracts.csv
Customer Engagements.csv
Customer Demo.csv


In [11]:
print(dfDict.keys())

# View a DataFrame 
display(dfDict['Customer Contracts$.csv'].head())

dict_keys(['Customer Contracts$.csv', 'customer_contracts.csv', 'Customer Engagements.csv', 'Customer Demo.csv'])


,customer_name,start_date,end_date,contract_amount_m,invoice_sent,paid
0,Nike,01-02-2019,12-20-2020,2.98,Yes,Yes
1,Reebox,06-20-2017,NaN,3.90,No,No
2,Adidas,12-07-2015,6-20-2018,4.82,Yes,Yes
3,Google,05-25-2014,03-20-2017,5.74,Yes,No
4,Amazon,11-10-2012,12-20-2015,6.66,No,Yes


### 5. Import CSV files to a postgreSQL database - automated script<a class="anchor" id="section5"></a>

In [12]:
for csvFile in csvFiles:
    
    DataFrame = dfDict[csvFile]

    ### 1. CLEAN TABLE & COLUMN HEADER NAMES
    
    # step 1. Cleans table (filename) names
    cleanTableName = csvFile.lower().replace('%', '').replace('$', '') \
                     .replace('£', '').replace('€', '').replace(')', '') \
                     .replace(r'(', '').replace('?', '').replace('!', '') \
                     .replace(' ', '_').replace('-', '_').replace(r'/', '_') \
                     .replace('\\', '_')
    
    # print(cleanTableName)
    
    # step 2. Remove the .csv extension from the filename
    finalTableName = '{0}'.format(cleanTableName.split('.')[0])
    
    # print(finalTableName)
    
    # step3. Cleans header names
    DataFrame.columns = [x.lower().replace('%', '').replace('$', '') \
                          .replace('£', '').replace('€', '').replace(')', '') \
                          .replace(r'(', '').replace('?', '').replace('!', '') \
                          .replace(' ', '_').replace('-', '_').replace(r'/', '_') \
                          .replace('\\', '_') for x in DataFrame.columns]
    
    # print(DataFrame.columns, '\n')
    
    ### 2. REPLACE PANDAS DTYPES WITH SQL DATABASE DTYPES
    
    # step 1. Dictionary mapping pandas dtypes to SQL dtypes
    dataTypeReplacements = {'object': 'varchar', 
                            'float64': 'float',
                            'int64': 'int', 
                            'datetime64': 'timestamp',
                            'timedelta64[ns]': 'varchar'}
    
    # step 2. Table schema string
    colString = ', '.join('{} {}'.format(n, d) for (n, d) in zip(DataFrame.columns, \
                                                                 DataFrame.dtypes.replace(dataTypeReplacements)))
    # print(colString)
    
    ### 3. OPEN A DATABASE CONNECTION
    
    # Database login credentials
    host = '####.cfa0pnoy####.eu-west-1.rds.amazonaws.com'
    databaseName = 'database_github'
    user = 'postgres'
    password = '####'
    
    # step 1. Database details string
    connectionString = "host=%s dbname=%s user=%s password=%s" % (host, databaseName, user, password)
    
    # step 2. Establish connection (logs in to database)
    rdsConnection = psycopg2.connect(connectionString)
    
    # step 3. Open the connection
    cursor = rdsConnection.cursor()
    print('Connected to database')
    
    ### 4. WRITE SQL STATEMENTS TO DATABASE
    
    # step 1. Drops any tables with the same name
    cursor.execute("DROP TABLE IF EXISTS %s;" % (finalTableName))
    
    # step 2. Creates table in the database
    cursor.execute("CREATE TABLE %s (%s)" % (finalTableName, colString))
    
    ### 5. INSERT VALUES INTO CREATED TABLE
    
    # step 1. Save pandas processed DataFrame to a csv file
    DataFrame.to_csv(csvFile, header=DataFrame.columns, index=False, encoding='utf-8')
    
    # step 2. Open the csv file to open it up in memory (note: file is saved as an object)
    myFile = open(csvFile)
    
    # step 3. Upload the csv file to the created table in database
    sqlStatement = """
                   COPY %s FROM STDIN WITH
                       CSV
                       HEADER
                       DELIMITER AS ','
                   """
    # step 4. Execute this SQL statement with copy_expert method - inserts values into the table
    cursor.copy_expert(sql=sqlStatement %finalTableName, file=myFile)
    
    ### TABLE PERMISSIONS
    
    # Grant multiple-users access
    cursor.execute('GRANT SELECT ON table %s TO public' % finalTableName)
    
    # Commit to database
    rdsConnection.commit()
    
    # Close the connection to database
    cursor.close()
    print('Table {0} successfully imported to database'.format(finalTableName))

Connected to database
Table customer_contracts successfully imported to database
Connected to database
Table customer_contracts successfully imported to database
Connected to database
Table customer_engagements successfully imported to database
Connected to database
Table customer_demo successfully imported to database


#### 5-1. Quick reference<a class="anchor" id="section5.1"></a>

In [13]:
# For loop - quick view
for csvFile in csvFiles:
    DataFrame = dfDict[csvFile]
    display(DataFrame)

,customer_name,start_date,end_date,contract_amount_m,invoice_sent,paid
0,Nike,01-02-2019,12-20-2020,2.98,Yes,Yes
1,Reebox,06-20-2017,NaN,3.90,No,No
2,Adidas,12-07-2015,6-20-2018,4.82,Yes,Yes
3,Google,05-25-2014,03-20-2017,5.74,Yes,No
4,Amazon,11-10-2012,12-20-2015,6.66,No,Yes
5,Facebook,04-29-2011,NaN,7.58,Yes,No
6,Apple,10-15-2009,NaN,8.50,Yes,Yes
7,Airbnb,04-02-2008,NaN,9.42,No,No
8,Nest,09-19-2006,NaN,3.00,Yes,Yes
9,Canon,03-07-2005,09-20-2009,11.26,Yes,No


,customer_name,start_date,end_date,contract_amount_m,invoice_sent,paid
0,Nike,01-02-2019,12-20-2020,2.98,Yes,Yes
1,Reebox,06-20-2017,NaN,3.90,No,No
2,Adidas,12-07-2015,6-20-2018,4.82,Yes,Yes
3,Google,05-25-2014,03-20-2017,5.74,Yes,No
4,Amazon,11-10-2012,12-20-2015,6.66,No,Yes
5,Facebook,04-29-2011,NaN,7.58,Yes,No
6,Apple,10-15-2009,NaN,8.50,Yes,Yes
7,Airbnb,04-02-2008,NaN,9.42,No,No
8,Nest,09-19-2006,NaN,3.00,Yes,Yes
9,Canon,03-07-2005,09-20-2009,11.26,Yes,No


,customer_id,num_of_users,_of_all_employees,sso,launched
0,101,10000,65%,Y,Yes
1,102,23423,80%,N,No
2,103,34556,30%,Y,Yes
3,104,123123,33%,N,No
4,105,19832,36%,Y,Yes
5,106,1243,75%,N,No
6,107,1231,42%,Y,Yes
7,108,12200,90%,N,No
8,109,200,48%,Y,Yes
9,110,1100,51%,N,No


,customer_id,customer_name,employee_count,office_location
0,101,Nike,120000,Oregon
1,102,Reebox,5000,California
2,103,Adidas,8000,CA
3,104,Google,500000,CA
4,105,Amazon,200000,Washington
5,106,Facebook,40002,CA
6,107,Apple,8000,CA
7,108,Airbnb,500000,CA
8,109,Nest,200000,CA
9,110,Canon,40002,NY
